# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import cassandra
import pandas as pd
import glob 
import datetime

#### Creating list of filepaths to process original event csv data files

In [2]:
# Write your code here
filepaths = []
path = 'youtube_event_data\\'
filepaths = glob.glob(path+'\\*.csv')

#### Processing the CSV files to create the new `youtube_events_data_new.csv` file

In [3]:
# Write your code here
youtube_events_data_new = []
# iterate through all files and store data inside list
for file in filepaths:
    data = pd.read_csv(file, index_col = None, header = 0)
    youtube_events_data_new.append(data)   
youtube_events_data_new = pd.concat(youtube_events_data_new, axis=0, ignore_index=True)
youtube_events_data_new.drop(['auth', 'gender', 'level', 'location', 'method', 'page', 'registration', 'status', 'ts'], axis=1, inplace=True)
youtube_events_data_new.to_csv('youtube_events_data_new.csv')
youtube_events_data_new.head()

,youtuber,firstName,itemInSession,lastName,length,sessionId,video,userId
0,NaN,Walter,0,Frye,NaN,38,NaN,39.0
1,NaN,Kaylee,0,Summers,NaN,139,NaN,8.0
2,Des'ree,Kaylee,1,Summers,246.30812,139,You Gotta Be,8.0
3,NaN,Kaylee,2,Summers,NaN,139,NaN,8.0
4,Mr Oizo,Kaylee,3,Summers,144.03873,139,Flat 55,8.0


In [4]:
# check the number of rows in your new csv file
youtube_events_data_new.shape[0]

8056

# Part II. Complete the Apache Cassandra coding portion of your project. 

**TODO: You have to complete the cell below with your `youtube_events_data_new.csv`columns and a screenshot of CSV file data**

## The new CSV file titled `youtube_events_data_new.csv`, The event_datafile_new.csv contains the following columns: 

- Column 1
- Column 2
- Column 3
- Column 4
.
.
.


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**youtube_events_data_new.csv**</font> after the code above is run:<br>

<img src="images/i.png">

#### Creating a Cluster

In [5]:
# Create a Cassandra Cluster
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace `youtubedb`

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS youtubedb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace `youtubedb`

In [7]:
try:
    session.set_keyspace('youtubedb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Give me the youtuber, video title and video's length in the YouTube app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of youtuber, video (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my YouTube app history who watched the video 'All Hands Against His Own'

### Creates table for each question

In [8]:
# table to answer question 1
query1 = "CREATE TABLE IF NOT EXISTS app_history_session_item"
query1 = query1 + "(sessionId int, itemInSession int, youtuber text, video text, length text, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

# table to answer question 2
query2 = "CREATE TABLE IF NOT EXISTS app_history_user_session "
query2 = query2 + "(userId int, sessionId int, itemInSession int, youtuber text, video text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
# table to answer question 3
query3 = "CREATE TABLE IF NOT EXISTS app_history_video "
query3 = query3 + "(video text, firstName text, lastName text, PRIMARY KEY (video, firstName, lastName))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [9]:
# read data from youtube_events_data_new.csv file
df = pd.read_csv('youtube_events_data_new.csv')
df['userId'] = df['userId'].fillna(0)

In [10]:
#insert data into first table
query1 = "INSERT INTO app_history_session_item (sessionId, itemInSession, youtuber, video, length)"
query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"

for i in range(df.shape[0]):
    data = (int(df['sessionId'].loc[i]), int(df['itemInSession'].loc[i]), str(df['youtuber'].loc[i]), str(df['video'].loc[i]), str(df['length'].loc[i]))
    try:
        session.execute(query1, data)
    except Exception as e:
        print(e)

#insert data into app_history_user_session table
query2 = "INSERT INTO app_history_user_session (userId, sessionId, itemInSession, youtuber, video, firstName, lastName)"
query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

for i in range(df.shape[0]):
    data = (int(df['userId'].loc[i]), int(df['sessionId'].loc[i]), int(df['itemInSession'].loc[i]), \
            str(df['youtuber'].loc[i]), str(df['video'].loc[i]), str(df['firstName'].loc[i]), str(df['lastName'].loc[i]))
    try:
        session.execute(query2, data)
    except Exception as e:
        print(e)
        
#insert data into app_history_video table
query3 = "INSERT INTO app_history_video (video, firstName, lastName)"
query3 = query3 + " VALUES (%s, %s, %s)"

for i in range(df.shape[0]):
    data = (str(df['video'].loc[i]), str(df['firstName'].loc[i]), str(df['lastName'].loc[i]))
    try:
        session.execute(query3, data)
    except Exception as e:
        print(e)

### Query 1: <br>
Give me the youtuber, video title and video's length in the YouTube app history that was heard during sessionId = 338, and   itemInSession = 4

In [11]:
try:
    rows = session.execute('SELECT * FROM app_history_session_item WHERE sessionId = 338 AND itemInSession = 4')
except Exception as e:
    print(e)
for row in rows:
    print ('Session Id: {}, iteminsession: {}, youtuber: {}, video: {}, length: {}'. \
           format(row.sessionid, row.iteminsession, row.youtuber, row.video, row.length))

Session Id: 338, iteminsession: 4, youtuber: Faithless, video: Music Matters (Mark Knight Dub), length: 495.3073


### Query 2: <br>
Give me only the following: name of youtuber, video (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
try:
    rows = session.execute('SELECT youtuber, video, firstName, lastName FROM app_history_user_session \
                           WHERE userId = 10 AND sessionId = 182')
except Exception as e:
    print(e)
for row in rows:
    print ('youtuber: {}, video: {}, first name: {}, last name: {}'. \
           format(row.youtuber, row.video, row.firstname, row.lastname))

youtuber: Down To The Bone, video: Keep On Keepin' On, first name: Sylvie, last name: Cruz
youtuber: Three Drives, video: Greece 2000, first name: Sylvie, last name: Cruz
youtuber: Sebastien Tellier, video: Kilometer, first name: Sylvie, last name: Cruz
youtuber: Lonnie Gordon, video: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), first name: Sylvie, last name: Cruz


### Query 3: <br>
Give me every user name (first and last) in my YouTube app history who watched the video 'All Hands Against His Own'

In [13]:
try:
    rows = session.execute("SELECT firstName, lastName FROM app_history_video WHERE video = 'All Hands Against His Own'")
except Exception as e:
    print(e)
    
for row in rows:
    print ('first name: {}, last name: {}'. \
           format(row.firstname, row.lastname))

first name: Jacqueline, last name: Lynch
first name: Sara, last name: Johnson
first name: Tegan, last name: Levine


### Drop the tables before closing out the sessions

In [14]:
query = "drop table app_history_session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table app_history_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table app_history_video"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()